<img src = "https://drive.google.com/uc?export=view&id=1HYrQHu1AE2cEid3BpNxo5KK3tuZF1TCo" alt = "Encabezado MLDS" width = "100%">  </img>

# **Análisis de datos**
---


## **0. Integrantes del equipo de trabajo**
---

Por favor incluya los nombres completos y número de identificación de los integrantes del equipo de trabajo:

1. 	Laura Alejandra Díaz López	1010018062
2. 	Diego Alejandro Feliciano Ramos	1024586904
3. 	Geraldine Gracia Ruiz	1032488268

In [ ]:
# Instalamos los paquetes necesarios para el notebook en caso de trabajar desde Google Colab
!pip install wget pandas ydata-profiling ipywidgets

In [ ]:
import wget, pandas as pd

# Descargar el dataset transformado de la entrega anterior del repositorio publico
url = 'https://github.com/MLDS-UN-ProjectTeam/final-term-project/raw/main/resources/data/Saber_11_2023-2_-_Preprocesado.csv?download='
downloaded_file_name = 'Saber_11_2023-2_-_Preprocesado.csv'
wget.download(url = url, out = downloaded_file_name)

# Cargar en el notebook el conjunto de datos con las restricciones conocidas:
# - Delimitador: ¬ 
# - Quoting = 3 => Sin comillas para todas las variables
saber11_dataframe = pd.read_csv(downloaded_file_name, delimiter='¬', quoting=3)

In [ ]:
# Importamos matplotlib y scipy para las visualizaciones.
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline
import numpy as np
from scipy import stats
import scipy
import statsmodels.api as sm

## **1. Análisis y resultados**
---

Al adentrarse en el análisis de datos, es crucial adoptar un enfoque estratégico que responda a las preguntas de planteadas en el proyecto. La selección de variables para estudio requiere una evaluación cuidadosa de los objetivos específicos del proyecto, priorizando aquellas que se consideren fundamentales para obtener resultados significativos.

A lo largo de esta entrega, trate busque responder las siguientes preguntas

- ¿Cómo se aplicaron conceptos específicos de estadística descriptiva e inferencial en el análisis de datos y cuál fue el enfoque utilizado para interpretar los resultados de manera precisa?
- ¿Cómo se interpretaron los resultados de la estadística descriptiva y qué conclusiones se pueden extraer?
- ¿Cómo se relacionan los resultados del análisis de datos con los objetivos y preguntas planteadas inicialmente?
- ¿Qué implicaciones prácticas tienen los hallazgos y cómo podrían influir en la toma de decisiones?

In [ ]:
# Observamos la información básica del dataframe.
saber11_dataframe.info()

Se empezó por comprobar que la variable objetivo de estudio, "PUNT_GLOBAL", estuviera normalmente distribuída, puesto que es una pieza de información importante a la hora de evaluar su correlación con otras variables. Esto se hizo a través de un Gráfico Q-Q.

In [ ]:
plt.figure(dpi = 110) # Mejoramos la visibilidad de la gráfica.
stats.probplot(saber11_dataframe['PUNT_GLOBAL'], plot= plt)
plt.title('Gráfico Q-Q del Puntaje Global')
plt.show()

<img src = "./resources/images/grafqq_puntglo.png" alt = "Gráfico Q-Q del Puntaje Global" width = "70%"></img>

A partir de la imagen es fácil observar que la mayoría de puntos azules, que representan a los datos, se encuentran sobre la línea roja, de donde se concluye que la variable de estudio SÍ está distribuída normalmente, con un leve comportamiento anormal en los cuantiles más bajo y más alto.

### **1.1. Análisis de correlaciones**
---

- Analice la posible existencia de relaciones entre las variables, utilizando medidas de correlación o visualizaciones pertinentes.
- Detecte la presencia de redundancias entre las variables, con el objetivo de facilitar su posterior eliminación o selección en el análisis.
- ¿Cuáles fueron las variables seleccionadas para el análisis de correlaciones, y cómo se eligieron en función de los objetivos del proyecto?



Para el análisis de correlaciones se hará uso de una función que permite visualizar la relación entre las dos variables.

In [ ]:
# Función para evaluar correlaciones entre variables

def graficar_corr(df, var_a, var_b):

  r = df[var_a].corr(df[var_b]);
  fig, ax = plt.subplots(dpi = 110);
  df.plot.scatter(var_a, var_b, title=f"Coeficiente de correlación: {r :.2}", ax = ax,
                marker='o', linestyle='None', s = 70, alpha = 0.5, grid=True 
                );


**Variables a evaluar:**

* FAMI_ESTRATOVIVIENDA_N : Estrato socioeconómico de la vivienda.
* ESTU_INSE_INDIVIDUAL : Índice Socioeconómico del evaluado
* ESTU_NSE_ESTABLECIMIENTO : Nivel Socioeconómico del Establecimiento
* FAMI_EDUCACIONMADRE : Nivel educativo más alto alcanzado por la madre.
* FAMI_EDUCACIONPADRE : Nivel educativo más alto alcanzado por el padre.

La selección de estas variables estuvo fundamentada en la limpieza y evaluación hecha en entregas anteriores. Adicionalmente se evaluó qué variables daban información más valiosa de acuerdo al objetivo principal del proyecto: encontrar factores socioeconómicos que afectan de forma relevante y determinante el rendimiento de los estudiantes a la hora de presentar el exámen de estado.

Se empezó entonces por las dos variables más evidentes, aquellas que da cuenta del estrato socioeconómico del estudiante; la primera, relacionada a su vivienda, la segunda, a su índice individual. Así:

In [ ]:
graficar_corr(saber11_dataframe, 'FAMI_ESTRATOVIVIENDA_N', 'PUNT_GLOBAL')

<img src = "./resources/images/corr_estrato_viv.png" alt = "Correlación entre el Putaje Global y el estrato socioeconómico de la vivienda del evaluado" width = "70%"></img>

In [ ]:
graficar_corr(saber11_dataframe, 'ESTU_INSE_INDIVIDUAL', 'PUNT_GLOBAL')

<img src = "./resources/images/corr_estu_inse.png" alt = "Correlación entre el Putaje Global y el Índice Socioeconómico del evaluado" width = "70%"></img>

De forma interesante, si bien se alcanzó a apreciar una correlación leve con el estrato socioeconómico de la vivienda del estudiante, fue al tener encuenta su índice individual que se hizo evidente el tipo de relación entre las dos variables. Se obtiene allí un coeficiente de correlación de 0.34 y, al visualizar los datos, se observó un resultado esperado: a mayor índice socioeconómico, mejores resultados en el exámen de estado. 

De forma similar se observó el nivel socioeconómico del establecimiento:

In [ ]:
graficar_corr(saber11_dataframe, 'ESTU_NSE_ESTABLECIMIENTO', 'PUNT_GLOBAL')

<img src = "./resources/images/corr_nse_establecimiento.png" alt = "Correlación entre el Putaje Global y el Nivel Socioeconómico del Establecimiento" width = "70%"></img>

Una vez más se hace obvia la diferencia: a medida que aumenta el nivel socioeconómico, mejoran los resultados del examen. Teniendo en cuenta que los tres resultados dan cuenta de factores similares y se puede incurrir en redundancias, se decidió tomar como parámetro de evaluación principal el Índice Individual Socioeconómico.  

Otro par de variables de interés se encuentran relacionadas con el nivel de estudios más alto alcanzado por la madre y el padre de cada estudiante, sin embargo, para poder hacer el análisis de correlaciones para estas variables fue necesario hacer una transformación de los datos, así:

In [ ]:
# RANGOS PARA NIVELES DE ESTUDIO
rangos_edu = {
    'Ninguno': 0,
    'No sabe': 0,
    'Primaria incompleta': 0.5,
    'Primaria completa': 1,
    'Secundaria (Bachillerato) incompleta': 1.5,
    'Secundaria (Bachillerato) completa': 2,
    'Técnica o tecnológica incompleta': 2.5,
    'Técnica o tecnológica completa': 3,
    'Educación profesional incompleta': 2.5,
    'Educación profesional completa': 4,
    'Postgrado': 5, 
}

In [ ]:
saber11_dataframe = saber11_dataframe.loc[saber11_dataframe['FAMI_EDUCACIONPADRE'] != 'No Aplica']
saber11_dataframe['FAMI_EDUCACIONPADRE'] = saber11_dataframe['FAMI_EDUCACIONPADRE'].map(rangos_edu)

In [ ]:
graficar_corr(saber11_dataframe, 'FAMI_EDUCACIONPADRE', 'PUNT_GLOBAL')

In [ ]:
saber11_dataframe = saber11_dataframe.loc[saber11_dataframe['FAMI_EDUCACIONMADRE'] != 'No Aplica']
saber11_dataframe['FAMI_EDUCACIONMADRE'] = saber11_dataframe['FAMI_EDUCACIONMADRE'].map(rangos_edu)

In [ ]:
graficar_corr(saber11_dataframe, 'FAMI_EDUCACIONMADRE', 'PUNT_GLOBAL')

<img src = "./resources/images/corr_edupadre.png" alt = "Correlación entre el Putaje Global y el nivel educativo más alto alcanzado por el padre" width = "70%"></img>

<img src = "./resources/images/corr_edumadre.png" alt = "Correlación entre el Putaje Global y el nivel educativo más alto alcanzado por el madre" width = "70%"></img>

Se observó aquí una correlación bastante pequeña pero, a pesar de tenue, existente, de donde se concluyó que a mayor nivel de estudios alcanzado por los padres, mejores resultados en el examen de estado. Dado que ambas variables tienen un comportamiento similar, se decidió elegir la que tiene mayor coeficiente de correlación, en este caso, la variable correspondiente a la madre. 

Tras el análisis de correlaciones entre varias variables de relevancia con el puntaje global obtenido por los estudiantes en el examen de estado, se concluyo que las variables más importantes a estudiar y tomar en cuenta son:

* Índice Individual de Nivel Socioeconómico. 
* Máximo nivel educativo alcanzado por la madre del estudiante. 

### **1.2. Análisis de regresiones**
---

- Apartir de las correlaciones identificadas, ¿es posible llevar a cabo un análisis de regresiones para estimar el valor de alguna de las variables de interés?
- ¿Cuáles fueron las variables seleccionadas para el análisis de regresiones y cómo se eligieron en función de los objetivos del proyecto?


Se eligieron las dos variables acorde a lo obtenido en el análisis de correlaciones, que además dan cuenta de forma amplia de la realidad socioeconómica del estudiante. 

Se empezó entonces por analizar el Índice Socioeconómico Indidivudal con una regresión lineal, usando la librería statsmodel para obtener información sobre el intercepto y la pendiente de la recta. 

In [ ]:
model = sm.OLS.from_formula("PUNT_GLOBAL ~ ESTU_INSE_INDIVIDUAL", data=saber11_dataframe)

result = model.fit()
result.summary()

<img src = "./resources/images/reg_dat_inse.png" alt = "Información de la regresión lineal del puntaje global en el examen de estado en función del índice socioeconómico indivudal" width = "70%"></img>

A partir de la información obtenida se procedió a visualizar el modelo lineal, así: 

In [ ]:
saber11_dataframe['modelo_INS'] =  2.3932 * saber11_dataframe['ESTU_INSE_INDIVIDUAL'] + 137.6087

ax = saber11_dataframe.plot.scatter('ESTU_INSE_INDIVIDUAL', 'PUNT_GLOBAL', title="Puntaje Global vs. Índice Socioeconómico Individual",
                s = 10, alpha = 0.3, grid=True)
saber11_dataframe.plot.line('ESTU_INSE_INDIVIDUAL', 'modelo_INS',
                   c='r', label='Regresión lineal', ax=ax);

<img src = "./resources/images/reg_graf_inse.png" alt = "Visualización de la regresión lineal del puntaje global en el examen de estado en función del índice socioeconómico indivudal" width = "70%"></img>

A pesar de la alta dispersión de los datos entorno a la recta y aunque el R^2 sea bastante bajo (0.120), la regresión ciertamente sigue el patrón comportamental de lo mismos.  

Se hizo el mismo procedimiento teniendo en cuenta el nivel máximo de estudios de la madre, así:

In [ ]:
model = sm.OLS.from_formula("PUNT_GLOBAL ~ FAMI_EDUCACIONMADRE", data=saber11_dataframe)

result = model.fit()
result.summary()

<img src = "./resources/images/reg_dat_edumadre.png" alt = "Información de la regresión lineal del puntaje global en función del nivel educativo más alto alcanzado por la madre" width = "70%"></img>

Así mismo para la visualización:

In [ ]:
saber11_dataframe['modelo_EDUMADRE'] =  10.5342 * saber11_dataframe['FAMI_EDUCACIONMADRE'] + 241.8596

ax = saber11_dataframe.plot.scatter('FAMI_EDUCACIONMADRE', 'PUNT_GLOBAL', title="Puntaje Global vs. Nivel de Estudios de la Madre",
                s = 10, alpha = 0.3, grid=True)
saber11_dataframe.plot.line('FAMI_EDUCACIONMADRE', 'modelo_EDUMADRE',
                   c='r', label='Regresión lineal', ax=ax);

<img src = "./resources/images/reg_graf_edumadre.png" alt = "Visualización de la regresión lineal del puntaje global en el examen de estado en función del nivel educativo más alto alcanzado por la madre" width = "70%"></img>

Se obtuvo aquí una pendiente mucho menos marcada a la encontrada para la otra variable, sin embargo, aún da cuenta de cómo a medida que el nivel educativo de la madre crece, también lo hace el puntaje del examen.

Para ambas variables se observó que a causa de la dispersión de los datos, no resulta tan sencillo estimar el valor de una variable a partir del modelo, sin embargo, este funciona muy bien para explicar el comportamiento que tienen las variables una respecto a la otra.

Se consideró la opción de estudiar esta variable categórica transformándola en dummies, así:

In [ ]:
import statsmodels.api as sm
from sklearn.model_selection import train_test_split

df_dummies = pd.get_dummies(saber11_dataframe['FAMI_EDUCACIONMADRE'], drop_first=True)
df_dummies = df_dummies.astype(int)
df_final = pd.concat([df_dummies, saber11_dataframe['PUNT_GLOBAL']], axis=1)
#print(df_final.dtypes)

X = df_final.drop('PUNT_GLOBAL', axis=1)
y = df_final['PUNT_GLOBAL'] 

X = sm.add_constant(X)

modelo = sm.OLS(y, X).fit()
print(modelo.summary())

### **1.3. Estadística inferencial (Opcional)**
---

- ¿Resulta pertinente calcular intervalos de confianza para estimar algún parámetro con confianza a partir de una muestra?
- ¿Es posible realizar pruebas de hipótesis para probar alguna teoría sobre un parámetro de la población?

In [ ]:
# ---INGRESE SU CÓDIGO---

---**INGRESE SU RESPUESTA**---

## **2. Resultados y visualizaciones**

### **2.1. Visualizaciones Estáticas**
---

* Incluya gráficos y visualizaciones estáticas que sean relevantes para los datos analizados.
* Explique cada visualización y cómo contribuye a la comprensión de los resultados.

In [ ]:
# ---INGRESE SU CÓDIGO---

### **2.2. Visualizaciones Interactivas**
---

* Cree visualizaciones interactivas, como gráficos dinámicos o tableros interactivos y proporcione instrucciones claras sobre cómo interactuar con ellos.
* Explore cómo estas visualizaciones ayudan en la exploración de datos y documente su análisis.

In [ ]:
# ---INGRESE SU CÓDIGO---

### **2.3. Visualización de Mapas Coropléticos (Opcional)**
---

* Si aplica mapas coropléticos para representar datos geoespaciales, incluya estas visualizaciones.
* Explique la elección de este tipo de visualización y cómo ésta contribuye a la comprensión de patrones geográficos y a la solución planteada.

In [ ]:
# ---INGRESE SU CÓDIGO---

### **2.4. Interpretación de Resultados**
---
* ¿Existen patrones o tendencias significativas en los datos?
* ¿Qué significan los resultados en términos prácticos?
* ¿Existen patrones o tendencias inesperadas en los datos que requieran una consideración especial o una investigación adicional?
* ¿Cuáles fueron los hallazgos principales que contribuirán al proceso de toma de decisiones?

Recuerde que la interpretación de resultados es una fase clave para extraer el significado de los datos y proporcionar respuestas o soluciones al problema inicial.

In [ ]:
# ---INGRESE SU CÓDIGO---

## **3. Conclusiones**
---
Asegúrese de incluir al menos una conclusión de cada unidad tratada en los *notebooks*. Estas conclusiones actúan como resúmenes breves que destacan los puntos clave y los aprendizajes esenciales de cada sección.


1.   
2.   
3.   
4.   



## **Créditos**
* **Profesor:** [Felipe Restrepo Calle](https://dis.unal.edu.co/~ferestrepoca/)
* **Asistentes docentes:**
    - [Juan Sebastián Lara Ramírez](https://www.linkedin.com/in/juan-sebastian-lara-ramirez-43570a214/).
* **Diseño de imágenes:**
    - [Rosa Alejandra Superlano Esquibel](mailto:rsuperlano@unal.edu.co).
* **Coordinador de virtualización:**
    - [Edder Hernández Forero](https://www.linkedin.com/in/edder-hernandez-forero-28aa8b207/).
    
**Universidad Nacional de Colombia** - *Facultad de Ingeniería*